## How do we convert words and sentences numbers and retaining their meaning?

- Computers can only understand numbers
- Let's look at several ways we can accomplish computerising words and sentences
  - [Co-occurence Matrix](https://colab.research.google.com/drive/1j3YPuv97Z-_bFhot2lIY7jpSfTjrrhi3#scrollTo=hLGshkOupgBq)
  - Term-Frequency
  - Term-Document
  - TF-IDF
  - Pointwise Mutual Information <br>
**Dense Vectors**<br>
  - SVDs 
  
  

## FAQ task

In this tuorial we'll describe how to build FAQ model based on config deeppavlov/configs/faq/tfidf_logreg_en_faq.json
<br>First of all we need train dataset of FAQ.

Data Source: https://www.sce.cornell.edu/ol/faq.php#1

E.g <br>
Q: Will classes meet at a specific time? <br>
A: With online learning, you may view the course materials on your own schedule. The content will be available to all students 24/7. However, you may be required to meet at set times with the faculty member.



       

![alt text](https://cdn-images-1.medium.com/max/600/1*e327fAqaxxifxcczLKwsdA.png)




**Note:** Please, install all necessary requirements using command:

>\>\> python -m deeppavlov install tfidf_logreg_en_faq.json

In [4]:
#Enable us store files "locally" google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

Mounted at /content/gdrive


In [0]:
# We use the ! to make for commandline commands

#In production/research settings, you want to focus on the tasks at hand. Not 
#installing and figuring our software dependencies. DevOps is a very good skill 
#to have but one should pick his fights wisely.
!pip install deeppavlov

     |████████████████████████████████| 686kB 2.8MB/s 
     |████████████████████████████████| 12.2MB 38.7MB/s 
     |████████████████████████████████| 51kB 17.5MB/s 
     |████████████████████████████████| 31.2MB 64.7MB/s 
     |████████████████████████████████| 61kB 22.6MB/s 
     |████████████████████████████████| 8.0MB 31.8MB/s 
     |████████████████████████████████| 307kB 43.8MB/s 
     |████████████████████████████████| 11.8MB 32.2MB/s 
     |████████████████████████████████| 3.4MB 32.0MB/s 
     |████████████████████████████████| 92kB 27.8MB/s 
     |████████████████████████████████| 51kB 21.7MB/s 
     |████████████████████████████████| 4.1MB 33.5MB/s 
     |████████████████████████████████| 12.4MB 41.1MB/s 
     |████████████████████████████████| 61kB 20.5MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 61kB 20.9MB/s 
     |████████████████████████████████| 143kB 39.7MB/s 
     |████████████████████████████████| 7.1MB 30.5MB/

In [0]:
!python -m deeppavlov install gdrive/My\ Drive/Colab\ Notebooks/IndabaXng/tfidf_logreg_en_faq.json

     |████████████████████████████████| 27.7MB 1.5MB/s 
     |████████████████████████████████| 2.1MB 31.1MB/s 
     |████████████████████████████████| 3.2MB 35.7MB/s 
     |████████████████████████████████| 17.3MB 35.9MB/s 
     |████████████████████████████████| 184kB 41.9MB/s 
ERROR: yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.1 which is incompatible.
ERROR: tensorflow 1.13.1 has requirement keras-applications>=1.0.6, but you'll have keras-applications 1.0.2 which is incompatible.
ERROR: tensorflow 1.13.1 has requirement keras-preprocessing>=1.0.5, but you'll have keras-preprocessing 1.0.1 which is incompatible.
ERROR: imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.1 which is incompatible.
ERROR: deeppavlov 0.3.0 has requirement numpy==1.14.5, but you'll have numpy 1.16.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incomp

In [0]:
import pandas as pd
FAQ_DATASET_URL = 'https://s3.amazonaws.com/mlnlpdatasets/cornell_faq30.csv'
faq_dataset = pd.read_csv(FAQ_DATASET_URL)
faq_dataset

In [0]:
import deeppavlov
from deeppavlov.models.tokenizers.spacy_tokenizer import StreamSpacyTokenizer
from deeppavlov.models.sklearn import SklearnComponent
# from deeppavlov.dataset_readers.faq_reader import FaqDatasetReader
from deeppavlov.core.data.data_learning_iterator import DataLearningIterator
from deeppavlov.core.data.utils import download_decompress


In [0]:
from pandas import read_csv
import numpy as np


# Our data might contain some missing values data
# In our case, we want to discard QA pairs missing any value.

data_unclean = faq_dataset
print(data_unclean.shape)
data_unclean['question'].replace('  ', np.nan, inplace=True)
data_unclean['answer'].replace('  ', np.nan, inplace=True)


data= data_unclean.dropna()
print(data.shape)
x = data['question']
y = data['answer']

train_xy_tuples = [(x.iloc[i], y.iloc[i]) for i in range(len(x))]

dataset = dict()
dataset["train"] = train_xy_tuples
dataset["valid"] = []
dataset["test"] = []

(1597, 3)
(1595, 3)


In [0]:
train_xy_tuples[2]

In [0]:
# Read FAQ data
# reader = FaqDatasetReader()
# faq_data = reader.read(data_url=FAQ_DATASET_URL, x_col_name='question', y_col_name='answer')
# iterator = DataLearningIterator(data=dataset)

# x,y = iterator.get_instances()

In [0]:
x[0]

## Train FAQ

Let's consider simple case for FAQ model (in the end you can find more complex pipeline models):
1. TF_IDF vectorizer on lemmatized questions
2. Logistic regression classifier

In [0]:
# create tokenizer
tokenizer = StreamSpacyTokenizer(lemmas=True)
x_tokenized = tokenizer(x)



x_tokenized

[['baby',
  'have',
  'catarrh',
  'and',
  'mild',
  'cough',
  'give',
  'vitamin',
  'c',
  'this',
  'morning',
  'and',
  'cover',
  'in',
  'warm',
  'water',
  'for',
  'some',
  'minute',
  'kindly',
  'advice',
  'what',
  'can',
  'use',
  'or',
  'do'],
 ['baby',
  'of',
  'and',
  'take',
  'opv',
  'on',
  'Saturday',
  'and',
  'some',
  'hour',
  'later',
  'start',
  'vomit',
  'and',
  'purge',
  'and',
  'take',
  'to',
  'hospital',
  'where',
  'be',
  'give',
  'zinc',
  'Para',
  'and',
  'pemadex',
  'injection',
  'a',
  'test',
  'to',
  'run',
  'the',
  'vomiting',
  'stop',
  'but',
  'on',
  'Sunday',
  'start',
  'purge',
  'water',
  'till',
  'now',
  'On',
  'Monday',
  'go',
  'back',
  'with',
  'the',
  'test',
  'result',
  'and',
  'ORS',
  'and',
  'coartem',
  'tab',
  'be',
  'give',
  'to',
  'be',
  'still',
  'stool',
  'water',
  'what',
  'do',
  'do',
  'stool',
  'be',
  'much'],
 ['old',
  'son',
  'pour',
  'angel',
  'baby',
  'powder'

In [0]:
x_tokens_joined = tokenizer(x_tokenized)
# fit TF-IDF vectorizer on train FAQ dataset 
vectorizer = SklearnComponent(model_class="sklearn.feature_extraction.text:TfidfVectorizer",
                              save_path='/content/gdrive/My Drive/Colab Notebooks/tfidf.pkl',
                              infer_method='transform')
X = vectorizer.fit(x_tokens_joined)

vocab = vectorizer.model.get_feature_names()
len(vocab)

In [0]:
# Now collect (x,y) pairs: x_train - vectorized question, y_train - answer from FAQ
x_train = vectorizer(x_tokens_joined)
y_train = y 

# Let's use top 2 answers for each incoming questions (top_n param)
clf = SklearnComponent(model_class="sklearn.linear_model:LogisticRegression",
                       top_n=2,
                       c=1000,
                       penalty='l2', 
                       save_path='/content/gdrive/My Drive/Colab Notebooks/tfidf_logreg_classifier_en_mipt_faq.pkl',
                       infer_method='predict')
clf.fit(x_train, y_train)

## Test FAQ

In [0]:
test_questions = ["begin my online class early"]
tokenized_test_questions = tokenizer(test_questions)
joined_test_q_tokens = tokenizer(tokenized_test_questions)
test_q_vectorized = vectorizer(joined_test_q_tokens)
answers = clf(test_q_vectorized)

Now we have all output of FAQ model: answers and scores.
<br>
Answers:

In [0]:
for i, answer in enumerate(answers):
    print('Answers {}:\n{}\n'.format(i, answer))

NameError: ignored

## Discussion/QA

- How can we evelaute our model's accuracy? BLUE?, LogisticRegression accuracy stats?
- How can we better our model?
- How else can we frame the problem?